### Imports

In [40]:
import pandas as pd
from pathlib import Path

### Load Data

In [187]:
ss = "/Users/tusharsingh/Library/CloudStorage/OneDrive-UniversityofSouthFlorida/self_practice/gatk/data/sample_sheet.tsv"

In [190]:
df = pd.read_csv(ss,usecols=["File Name","Case ID","Sample ID","Tissue Type","Tumor Descriptor"],sep='\t')
### Filter
df['Tissue Type'] =  df['Tissue Type'] .str.replace(",.*",'',regex=True)
df['Case ID']     =  df['Case ID']     .str.replace(",.*",'',regex=True)
### Remove them later
myset  = set(df[df['Tumor Descriptor']=='Recurrence']['Case ID'])
### extra filter
df = df[(df['Tumor Descriptor']!='Recurrence') & (~df['Sample ID'].str.contains('^SM')) ]
df.sort_values('Case ID').head()

,File Name,Case ID,Sample ID,Tissue Type,Tumor Descriptor
440,aee72b24-81c3-47b5-856d-35ff22208f84_wxs_gdc_r...,C1230738,1105198,Tumor,Primary
438,50a02e53-a4d7-4e01-9505-1321cc9f0e32_wxs_gdc_r...,C1230738,1105281,Normal,Not Applicable
367,20c88342-81c8-4764-9735-20ad94c112dc_wxs_gdc_r...,C1245129,1105282,Normal,Not Applicable
147,fb858b07-76dc-48b4-9096-adb0aed38c31_wxs_gdc_r...,C1245129,1105251,Tumor,Primary
307,881dda1b-14e9-4952-875d-033de239cc69_wxs_gdc_r...,C204057,1105284,Normal,Not Applicable


In [192]:
test = df.groupby(['Case ID','Tissue Type'])['File Name'].first().reset_index()
test = test[~test['Case ID'].isin(myset)]

In [193]:
test['Tissue Type'].value_counts()

Tissue Type
Normal    176
Tumor     175
Name: count, dtype: int64

In [201]:
set(test[test['Tissue Type']=='Normal']['Case ID'].to_list()) - set(test[test['Tissue Type']=='Tumor']['Case ID'].to_list())

{'C3N-01197'}

In [ ]:
test[test['Case ID']=='C3N-01197']

In [206]:
test = test[test['Case ID']!='C3N-01197'].copy()

In [210]:
test.value_counts('Tissue Type')

Tissue Type
Normal    175
Tumor     175
Name: count, dtype: int64

In [211]:
set(test[test['Tissue Type']=='Normal']['Case ID'].to_list()) - set(test[test['Tissue Type']=='Tumor']['Case ID'].to_list())

set()

In [212]:
test

,Case ID,Tissue Type,File Name
31,C3L-00104,Normal,27b12d40-946d-4fc4-b336-b0edd9bcaa3c_wxs_gdc_r...
32,C3L-00104,Tumor,d36cf415-f716-4e50-a949-a06459cd5277_wxs_gdc_r...
33,C3L-00365,Normal,1ed64831-3495-49f5-a860-b775ad3586e9_gdc_realn...
34,C3L-00365,Tumor,fded09b0-7908-4f9f-9f43-b3e8833bf39e_gdc_realn...
35,C3L-00674,Normal,7a56ec48-005d-401e-b8fc-30a60472483d_gdc_realn...
...,...,...,...
377,C3N-05114,Tumor,5169afe8-1fb1-459b-b5b6-f28129bc1de1_wxs_gdc_r...
378,C3N-05135,Normal,d69369cc-a994-4e4f-af13-4948398b347b_wxs_gdc_r...
379,C3N-05135,Tumor,f00c2f88-4ae1-4bfc-8d04-8f2c4daa2515_wxs_gdc_r...
380,C3N-05991,Normal,099752ff-7fa5-4eb0-831a-09e5a211d5c3_wxs_gdc_r...


In [241]:
main =test.pivot_table(values  ='File Name',
                 index   ='Case ID',
                 columns = 'Tissue Type',
                 aggfunc='first').reset_index()

In [240]:
test.pivot_table(values  ='File Name',
                 index   ='Case ID',
                 columns = 'Tissue Type',
                 aggfunc='first').reset_index()

Tissue Type,Case ID,Normal,Tumor
0,C3L-00104,27b12d40-946d-4fc4-b336-b0edd9bcaa3c_wxs_gdc_r...,d36cf415-f716-4e50-a949-a06459cd5277_wxs_gdc_r...
1,C3L-00365,1ed64831-3495-49f5-a860-b775ad3586e9_gdc_realn...,fded09b0-7908-4f9f-9f43-b3e8833bf39e_gdc_realn...
2,C3L-00674,7a56ec48-005d-401e-b8fc-30a60472483d_gdc_realn...,6d3bba42-a739-401a-b95c-44a01c7780c0_gdc_realn...
3,C3L-00677,5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn...,1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn...
4,C3L-01040,30d0d5fe-c267-4b3a-b5a0-b634438c5f22_wxs_gdc_r...,82659d5a-a5f5-43bd-86ed-b64dcc4d4503_wxs_gdc_r...
...,...,...,...
170,C3N-04695,dc406b2d-e0c8-49bf-b1b1-7e1929833382_wxs_gdc_r...,d7f885b4-11bb-4eef-81bf-74643b30908c_wxs_gdc_r...
171,C3N-05039,e1965bd7-ba72-47a9-b81d-d42f2b1cf4c0_wxs_gdc_r...,fe17632c-e4bd-4c42-aa43-255206b1a97b_wxs_gdc_r...
172,C3N-05114,e8446ae9-fcf6-48e3-8073-0b0ba6396588_wxs_gdc_r...,5169afe8-1fb1-459b-b5b6-f28129bc1de1_wxs_gdc_r...
173,C3N-05135,d69369cc-a994-4e4f-af13-4948398b347b_wxs_gdc_r...,f00c2f88-4ae1-4bfc-8d04-8f2c4daa2515_wxs_gdc_r...


In [274]:
sum(test.groupby('Case ID')['Tissue Type'].apply(lambda x: 
     (x.eq('Tumor').sum() > 0) and (x.eq('Normal').sum() > 0) ))       

                             

175

In [269]:
test['Case ID'].eq('Tumor')

31     False
32     False
33     False
34     False
35     False
       ...  
377    False
378    False
379    False
380    False
381    False
Name: Case ID, Length: 350, dtype: bool

In [273]:
# main.to_csv('/Users/tusharsingh/Library/CloudStorage/OneDrive-UniversityofSouthFlorida/self_practice/gatk/data/sample_pairs.txt',index=False,header=False,mode="a")

Case ID
C3L-00104    True
C3L-00365    True
C3L-00674    True
C3L-00677    True
C3L-01040    True
             ... 
C3N-04695    True
C3N-05039    True
C3N-05114    True
C3N-05135    True
C3N-05991    True
Name: Tissue Type, Length: 175, dtype: bool

In [ ]:
x.rese